<a href="https://colab.research.google.com/github/gannannavelorange/575/blob/master/src/ib_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
!git clone https://github.com/gannannavelorange/575

Cloning into '575'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 436 (delta 25), reused 0 (delta 0), pack-reused 382
Receiving objects: 100% (436/436), 1.91 MiB | 4.06 MiB/s, done.
Resolving deltas: 100% (163/163), done.


In [64]:
cd 575/src/data_IB_project

/content/575/src/data_IB_project/575/src/data_IB_project/575/src/data_IB_project


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import scipy.stats as ss
from scipy.stats import norm
from scipy.stats import t

In [0]:
'''
#get weekly interest rate change factor for coupon bond
def getdata(M,N):
  with open(M+'.csv') as file:
      reader=csv.reader(file)
      rows=[row for row in reader]
      DD=[row[1] for row in rows[1:293]]
      DD=[float(d)/100/2 for d in DD]
      D=[row[2] for row in rows[1+N:250+N]]
      D=[float(d)/100/2 for d in D]
      return DD,D
      #print(D)
      #print(D[0])


In [0]:
#get weekly interest rate change factor for coupon bond
def getdata(M,N):
  with open(M+'.csv') as file:
      reader=csv.reader(file)
      rows=[row for row in reader]
      DD=[row[1] for row in rows[1:62]]
      DD=[float(d)/100/2 for d in DD]     #continuous compounded interest rate 
      D=[row[2] for row in rows[1+N:53+N]]
      D=[float(d)/100/2 for d in D]   #risk factor change
      return DD,D

In [33]:
'''
test1=getdata('bond1',0)
kk = t.fit(test1[1])
print(kk[0])
print(kk[1])
print(kk[2])
nn=norm.fit(test1[1])
print(nn[0])
print(nn[1])
E=np.mean(test1[1])
print(E)

2.0272787924820648
-0.00010279419664695968
0.00025628701298503385
-8.60576923076923e-05
0.000381080620052096
-8.60576923076923e-05


In [0]:
'''
g=getdata('20J28',0)[0]
print(g)
ks = norm.fit(g)
print(ks[0],ks[1])


In [0]:

# for each coupon, linloss could be calculated as L=-c*np.exp(r*t)(r*5-t*x) week=5 days
def linloss_bond(c,r,t,mu,sigma):
  exp=-c*np.exp(r*t)*(r*5-t*mu)
  #exp=-c*np.exp(r*t)*(r*1-t*mu)
  var=(np.exp(r*t)*t)**2*(sigma**2)
  return exp,var

In [0]:

def rolling_bond(day,F,c,r,mu,sigma):
  Exp=-F*np.exp(r*(day/182+15))*(r*5-(day/182+15)*mu)
  Var=(np.exp(r*(day/182+15))*(day/182+15))**2*(sigma**2)
  for i in range(0,16):
    t=day/182+i
    Exp=Exp+linloss_bond(c,r,t,mu,sigma)[0]
    Var=Var+linloss_bond(c,r,t,mu,sigma)[1]
  return Exp,Var


In [0]:
c1=1.75/100*1000/2
c2=6/100*1000/2
F1=1000

In [0]:
Bond1_mu=[]
Bond1_var=[]
Bond2_mu=[]
Bond2_var=[]
for n in range(0,10):   # estimate 2010/02/14-2020/04/17
 dd_1=getdata('bond1',n)[0]
 d_1=getdata('bond1',n)[1]
 ks_1 = norm.fit(d_1)
 r1=dd_1[51+n] 
 dd_2=getdata('bond3.625',n)[0]
 d_2=getdata('bond3.625',n)[1]
 ks_2 = norm.fit(d_2)
 r2=dd_2[51+n] 
 week_bond1=rolling_bond(159-7*n,F1,c1,r1,ks_1[0],ks_1[1])   #159=2020/07/15-2020/02/14
 Bond1_mu.append(week_bond1[0])
 Bond1_var.append(week_bond1[1])
 week_bond2=rolling_bond(182-7*n,F1,c2,r2,ks_2[0],ks_2[1])  
 Bond2_mu.append(week_bond2[0])
 Bond2_var.append(week_bond2[1])
 #print(week[0],week[1])
 #print(week[0])


In [19]:
print(Bond1_mu)
print(Bond1_var)
print(Bond2_mu)
print(Bond2_var)

[1.3168020657115707, 3.289005837349064, 6.371792178381435, 11.77538788766703, -9.248422056742587, -17.586704545032735, 1.294661141562513, 7.12953190755665, 8.49477900869689, 5.478700272366696]
[0.0002457889910209126, 0.00024171508720053275, 0.00023729352080909715, 0.0002525068626788285, 0.0008218318228104392, 0.0009150931923024108, 0.0011847284904108561, 0.0011737333097386084, 0.001158816055744269, 0.0011802863708535824]
[0.5195273966302107, 2.744067360238744, 7.740156633816925, 14.924643187375043, -12.414439799099899, -23.05119146725257, -0.3601595108727958, 7.499844908180267, 8.863422497565818, 5.077241701871107]
[0.0002530083766022384, 0.0002486028992484554, 0.0002481800849339859, 0.00026476457259763073, 0.0008406554152650169, 0.0009338283838282592, 0.0011618050204984934, 0.0011540053541670703, 0.001140687930817159, 0.0011602265035315596]


In [0]:
#put option

In [0]:
def N(x):
  cdf=norm.cdf(x,0,1)
  return cdf
def p(x):
  pdf=norm.pdf(x,0,1)
  return pdf

In [0]:
#get weekly stock price log return for option
def getdata2(M,N):
  with open(M+'.csv') as file:
      reader=csv.reader(file)
      rows=[row for row in reader]
      DD=[row[1] for row in rows[1:62]]
      DD=[float(d) for d in DD]
      D=[row[2] for row in rows[1+N:53+N]]
      D=[float(d) for d in D]
      net=[row[3] for row in rows[1+N:53+N]]
      net=[float(d) for d in net]
      return DD,D,net

In [152]:
'''
djx=getdata2('DIA',0)
djx1=djx[0]
djx2=djx[1]
std=np.std(djx2)
print(djx1[50])


289.362


In [0]:
v_djx=[9.5,9,11.21,39.96,38.84,64.99,102.26,78.84,83.26,57.34,51.95] 
ir=[-0.07,-0.15,-0.28,-0.57,0.04,0.17,-0.22,-0.46,-0.5,-0.36]


In [0]:
'''
def d(S,K,r,sigma,t):
 d1=(np.log(S/K)+(r+sigma**2/2)*t)/(sigma*np.sqrt(t))
 d2=d1-sigma*np.sqrt(t)
 return d1,d2

In [0]:
def deltaput(S,K,r,sigma,t):
  d1=(np.log(S/K)+(r+sigma**2/2)*t)/(sigma*np.sqrt(t))
  delta_P=N(float(d1))-1
  return delta_P

In [62]:
djxmu=[]
djxvariance=[]
for i in range(1,10):    #estimate loss on 2020/02/14 02/21 02/28 3/6
  djx=getdata2('DIA',i)
  djx1=djx[0]   #price
  djx2=djx[1]    #log return
  djx3=djx[2]   #net return of stock
  std=np.std(djx2)
  delta=deltaput(djx1[51+i],293,ir[i-1],std,(125-7*i)/250)   #t=(2020/06/18-2020/02/14)/250
  leverage=delta*djx1[51+i-1]/v_djx[i-1]
  djxmu.append(leverage*djx1[51+i]*np.mean(djx2))     #loss:expected mean_option=L*expected mean_stock=L*S_t*mean(log return)
  djxvariance.append(leverage**2*(std**2))       #loss:expected variance_option=L^2*expected variance_stock=L^2*(S_t^2)*(sigma^2)
  #print(delta)
  #print(leverage)
print(djxmu)
print(djxvariance)

[-22.208631975923012, 0.09049563621045381, -4.299216043087048, 2.4978428937689805, 5.734177888341507, 1.9312491289532265, 1.7420866749395347, 1.028307751491843, 0.9039767293429295]
[0.21836168115661, 0.5747432849919032, 0.2835965729756227, 0.03229678699657972, 0.05048359118705913, 0.014903149391282079, 0.007654152201570178, 0.0142229465367903, 0.016250890396684325]


In [0]:
#using WEEKLY log return
mu_m=np.zeros(15)
names_2=['CAT','DIA','HD','MSFT','NKE','NVDA','PG','TM','VIG','VOO','WMT','GOOG','CHEF','SPB','ORCL']
def getData_m(n=None):
    Is=[i for i in range(15)]
    Rows=[]
    for name,i in zip(names_2,Is):
        with open(name+'.csv') as file:
            reader=csv.reader(file)
            rows=[row[2] for row in reader]
        mu_m[i]=np.mean([float(row) for row in rows[1+N:53+N]])
        Rows.append([float(row) for row in rows[1+N:53+N]])
    Omega_m=np.cov(np.array(Rows))
    return mu_m,Omega_m

In [0]:
#linearized loss mean&variance
def lin_loss(Vt,Wt,mu,omega):
  exp=-Vt*(np.dot(Wt,mu))
  var=(Vt**2)*(np.dot(np.dot(Wt,omega),Wt))
  return exp,var

In [0]:
'''
#rolling window for linearized loss using WEEKLY log return 
for M in range(10):
  data3_m= getData_m(M+1)
  mu3_m=data3_m[0]
  omega3_m=data3_m[1]
  vt=PORT_VAL[M]
  a3_m=lin_loss(vt,wt,mu3_m,omega3_m)
  #print(a3_m[0])
  #print(a3_m[1])
  x = np.linspace (a3_m[0]-4*np.sqrt(a3_m[1]), a3_m[0]+4*np.sqrt(a3_m[1]))
  y = normfun(x, a3_m[0], np.sqrt(a3_m[1]))
  plt.plot(x,y)
  plt.title('PDF of linearized loss distribution using four year rolling window')

In [0]:
#linearized loss using monthly log return
mu_m=np.array(getData_m(49)[0])    #first four year monthly log return
omega_m=np.array(getData_m(49)[1])
#print(omega)
vt=193.664                           #20151231 portfolio value
wt=np.ones(15)*1/15
b=lin_loss(vt,wt,mu_m,omega_m)
exp_m=b[0]
var_m=b[1]
print(exp_m)
print(var_m)

In [0]:
#VaR
def VaRnorm():
  VaR=mu+sigma*norm.ppf(alpha)
  ES=mu+sigma*norm.pdf(norm.ppf(alpha))/(1-alpha)
  return VaR,ES

In [0]:
def VaRt():
  VaR=mu+sigma*t.ppf(alpha)
  ES=mu+sigma*t.pdf(t.ppf(alpha))/(1-alpha)*(v+(t.ppf(alpha))**2)/(v-1)
  return VaR,ES

In [37]:

a=norm.pdf(0.5)
b=p(0.5)
print(b)

0.3520653267642995
